# Homework 6

## CSCI E-82A


In the lesson we constructed a representation of a simple grid world. Dynamic programming (DP) was used to find optimal plans for a robot to navigate from any starting location on the grid to the goal. This problem is an analog for more complex real-world robot navigation problems. 

In this homework you will use DP to solve a slightly more complex robotic navigation problem in a grid world. This grid world is a simple version of the problem a material transport robot might encounter in a warehouse. The situation is illustrated in the figure below.

<img src="GridWorldFactory.JPG" alt="Drawing" style="width:200px; height:200px"/>
<center> **Grid World for Factory Navigation Example** </center>

### Reference:

- [Gridworld - Stanford](https://cs.stanford.edu/people/karpathy/reinforcejs/gridworld_dp.html)
- [Grid - Python](https://www.hackerearth.com/fr/practice/notes/dynamic-programming-problems-involving-grids/) 
- [Python Optimization](https://medium.com/@annishared/searching-for-optimal-policies-in-python-an-intro-to-optimization-7182d6fe4dba) 

The goal is for the robot to deliver some material to position (state) 12, shown in blue. Since there is a goal state or **terminal state** this an **episodic task**. 

There are some barriers comprised of the states $\{ 6, 7, 8 \}$ and $\{ 16, 17, 18 \}$, shown with hash marks. In a real warehouse, these positions might be occupied by shelving or equipment. We do not want the robot to hit these barriers. Thus, we say that transitioning to these barrier states is **taboo**.

As before, we do not want the robot to hit the edges of the grid world, which represent the outer walls of the warehouse. 

## Representation

As with many such problems, the starting place is creating the **representation**. In the cell below encode your representation for the possible action-state transitions. From each state there are 4 possible actions:
- up, u
- down, d,
- left, l
- right, r

There are a few special cases you need to consider:
- Any action transitioning state off the grid or into a barrier should keep the state unchanged. 
- Any action in the goal state keeps the state unchanged. 
- Any transition within the taboo (barrier) states can keep the state unchanged. If you experiment, you will see that other encodings work as well since the value of a barrier states are always zero and there are no actions transitioning into these states. 

> **Hint:** It may help you create a pencil and paper sketch of the transitions, rewards, and probabilities or policy. This can help you to keep the bookkeeping correct. 

In [1]:
import numpy as np

policy = {0:{'u':0, 'd':5, 'l':0, 'r':1},
          1:{'u':1, 'd':1, 'l':0, 'r':2},
          2:{'u':2, 'd':2, 'l':1, 'r':3},
          3:{'u':3, 'd':3, 'l':2, 'r':4},
          4:{'u':4, 'd':9, 'l':3, 'r':4},
          5:{'u':0, 'd':10, 'l':5, 'r':5},
          6:{'u':0, 'd':0, 'l':0, 'r':0},
          7:{'u':0, 'd':0, 'l':0, 'r':0},
          8:{'u':0, 'd':0, 'l':0, 'r':0},
          9:{'u':4, 'd':14, 'l':9, 'r':9},
          10:{'u':5, 'd':15, 'l':10, 'r':11},
          11:{'u':11, 'd':11, 'l':10, 'r':12},
          12:{'u':0, 'd':0, 'l':0, 'r':0},
          13:{'u':13, 'd':13, 'l':12, 'r':14},
          14:{'u':9, 'd':19, 'l':13, 'r':14},
          15:{'u':10, 'd':20, 'l':15, 'r':15},
          16:{'u':0, 'd':0, 'l':0, 'r':0},
          17:{'u':0, 'd':0, 'l':0, 'r':0},
          18:{'u':0, 'd':0, 'l':0, 'r':0},
          19:{'u':14, 'd':24, 'l':19, 'r':19},
          20:{'u':15, 'd':20, 'l':20, 'r':21},
          21:{'u':21, 'd':21, 'l':20, 'r':22},
          22:{'u':22, 'd':22, 'l':21, 'r':23},
          23:{'u':23, 'd':23, 'l':22, 'r':24},
          24:{'u':19, 'd':24, 'l':23, 'r':24}}

You need to define the initial transition probabilities for the Markov process. Set the probabilities for each transition as a **uniform distribution** leading to random action by the robot. 

> **Note:** As these are just starting values, the exact values of the transition probabilities are not actually all that important in terms of solving the DP problem. Also, notice that it does not matter how the taboo state transitions are encoded. The point of the DP algorithm is to learn the transition policy.

In [2]:
state_to_state_probs = {0:{'u':0.25, 'd':0.25, 'l': 0.25, 'r':0.25},
                      1:{'u':0.25, 'd':0.25, 'l': 0.25, 'r':0.25},
                      2:{'u':0.25, 'd':0.25, 'l': 0.25, 'r':0.25},
                      3:{'u':0.25, 'd':0.25, 'l': 0.25, 'r':0.25},
                      4:{'u':0.25, 'd':0.25, 'l': 0.25, 'r':0.25},
                      5:{'u':0.25, 'd':0.25, 'l': 0.25, 'r':0.25},
                      6:{'u':0.25, 'd':0.25, 'l': 0.25, 'r':0.25},
                      7:{'u':0.25, 'd':0.25, 'l': 0.25, 'r':0.25},
                      8:{'u':0.25, 'd':0.25, 'l': 0.25, 'r':0.25},
                      9:{'u':0.25, 'd':0.25, 'l': 0.25, 'r':0.25},
                      10:{'u':0.25, 'd':0.25, 'l': 0.25, 'r':0.25},
                      11:{'u':0.25, 'd':0.25, 'l': 0.25, 'r':0.25},
                      12:{'u':0.0, 'd':0.0, 'l':0.0, 'r':0.0},
                      13:{'u':0.25, 'd':0.25, 'l': 0.25, 'r':0.25},
                      14:{'u':0.25, 'd':0.25, 'l': 0.25, 'r':0.25},
                      15:{'u':0.25, 'd':0.25, 'l': 0.25, 'r':0.25},
                      16:{'u':0.25, 'd':0.25, 'l': 0.25, 'r':0.25},
                      17:{'u':0.25, 'd':0.25, 'l': 0.25, 'r':0.25},
                      18:{'u':0.25, 'd':0.25, 'l': 0.25, 'r':0.25},
                      19:{'u':0.25, 'd':0.25, 'l': 0.25, 'r':0.25},
                      20:{'u':0.25, 'd':0.25, 'l': 0.25, 'r':0.25},
                      21:{'u':0.25, 'd':0.25, 'l': 0.25, 'r':0.25},
                      22:{'u':0.25, 'd':0.25, 'l': 0.25, 'r':0.25},
                      23:{'u':0.25, 'd':0.25, 'l': 0.25, 'r':0.25},
                      24:{'u':0.25, 'd':0.25, 'l': 0.25, 'r':0.25}}

The robot receives the following rewards:
- +10 for achieving the goal. 
- -1 for attempting to leave the warehouse or hitting the barriers. In other words, we penalize the robot for hitting the edges of the grid or the barriers.  
- -0.1 for all other state transitions, which is the cost for the robot to move from one state to another.  

In the code cell below encode your representation of this reward structure.  

In [3]:
rewards =  {0:{'u':-1, 'd':-0.1, 'l':-1, 'r':-0.1},
          1:{'u':-1, 'd':-1, 'l':-0.1, 'r':-0.1},
          2:{'u':-1, 'd':-1, 'l':-0.1, 'r':-0.1},
          3:{'u':-1, 'd':-1, 'l':-0.1, 'r':-0.1},
          4:{'u':-1, 'd':-0.1, 'l':-0.1, 'r':-1},
          5:{'u':-0.1, 'd':-0.1, 'l':-1, 'r':-1},
          6:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          7:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          8:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          9:{'u':-0.1, 'd':-0.1, 'l':-1, 'r':-1},
          10:{'u':-0.1, 'd':-0.1, 'l':-1, 'r':-0.1},
          11:{'u':-1, 'd':-1, 'l':-0.1, 'r':10},
          12:{'u':0.0, 'd':0.0, 'l':0.0, 'r':0.0},
          13:{'u':-1, 'd':-1, 'l':10, 'r':-0.1},
          14:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-1},
          15:{'u':-0.1, 'd':-0.1, 'l':-1, 'r':-1},
          16:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          17:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          18:{'u':-0.1, 'd':-0.1, 'l':-0.1, 'r':-0.1},
          19:{'u':-0.1, 'd':-0.1, 'l':-1, 'r':-1},
          20:{'u':-0.1, 'd':-1, 'l':-1, 'r':-0.1},
          21:{'u':-1, 'd':-1, 'l':-0.1, 'r':-0.1},
          22:{'u':-1, 'd':-1, 'l':-0.1, 'r':-0.1},
          23:{'u':-1, 'd':-1, 'l':-0.1, 'r':-0.1},
          24:{'u':-0.1, 'd':-1, 'l':-0.1, 'r':-1}}

You will find it useful to create a list of taboo states, which you can encode in the cell below.

In [4]:
taboos = [6, 7, 8, 16, 17, 18]
#taboos = []

## Policy Evaluation

With your representation defined, you can now create and test a function for **policy evaluation**. You will need this function for your policy iteration code. 

You are welcome to state with the `compute_state_value` function from the DP notebook. However, keep in mind that you must modify this code to correctly treat the taboo states. Specifically, taboo states should have 0 value. 

In [ ]:
def compute_state_value(pi, probs, reward, taboos, gamma = 1.0, theta = 0.01, display = False):
    '''Function for policy evaluation  
    '''
    delta = theta
    values = np.zeros(len(probs)) # Initialize the value array
    while(delta >= theta):
        v = np.copy(values) ## save the values for computing the difference later
        for s in probs.keys():
            temp_values = 0.0 ## Initial the sum of values for this state
            if s not in taboos: ## Adding a condition to test for taboos 
                for action in rewards[s].keys():
                    s_prime = pi[s][action]
                    temp_values = temp_values + probs[s][action] * (reward[s][action] + gamma * values[s_prime])
            values[s] = temp_values
            
        ## Compute the difference metric to see if convergence has been reached.    
        diffs = np.sum(np.abs(np.subtract(v, values)))
        delta = min([delta, diffs])
        if(display): 
            print('difference metric = ' + str(diffs))
            print(values.reshape(5,5))
    return values

compute_state_value(policy, state_to_state_probs, rewards, taboos, theta = 0.1, display = False).reshape(5,5)

array([[-38.28926029, -41.56189427, -42.65499935, -41.56857879,
        -38.30169862],
       [-32.84169421,   0.        ,   0.        ,   0.        ,
        -32.8525447 ],
       [-25.20972897,  -8.65258033,   0.        ,  -8.65482963,
        -25.21851918],
       [-32.84716098,   0.        ,   0.        ,   0.        ,
        -32.85784594],
       [-38.3016983 , -41.5751609 , -42.66847476, -41.58164302,
        -38.31375999]])

Examine the state values you have computed using a random walk for the robot. Answer the following questions:

1. Are the values of the goal and taboo states zero? ANS: 
2. Do the values of the states increase closer to the goal? ANS: 
3. Do the goal and barrier states all have zero values? ANS: 

If your answer to any of the above questions is no, you need to do some more work on your code!

## Policy Iteration

Now that you have your representation and a function to perform policy evaluation you have everything you need to use the policy iteration algorithm to create an optimal policy for the robot to reach the goal. 

If your policy evaluation function works correctly, you should be able to use the `policy_iteration` function from the DP notebook. Make sure you print the state values as well as the policy you discovered. 

In [ ]:
import copy
def policy_iteration(pi, probs, reward, taboos, gamma = 1.0, theta = 0.1, output = False):
    delta = theta
    v = np.zeros(len(probs))
    state_values = np.zeros(len(probs))
    current_policy = copy.deepcopy(probs)
    while(delta >= theta): # Continue until convergence.
        for s in probs.keys(): # Iterate over all states
            temp_values = []
            for action in rewards[s].keys(): # Iterate over all possible actions for the state
                # Compute list of values given action from the state
                s_prime = pi[s][action]
                temp_values.append(current_policy[s][action] * (reward[s][action] + gamma * state_values[s_prime]))
            
            ## Find the max value and update current policy
            max_index = np.where(np.array(temp_values) == max(temp_values))[0]
            prob_for_policy = 1.0/float(len(max_index))
            for i,action in enumerate(current_policy[s].keys()): 
                if(i in max_index): current_policy[s][action] = prob_for_policy
                else: current_policy[s][action] = 0.0
                
        
        ## Compute state values with new policy to determine if there is an improvement
        ## Uses the compute_state_value function
        state_values = compute_state_value(pi, current_policy, rewards, taboos, theta = .1)
        diff = np.sum(np.abs(np.subtract(v, state_values)))
        if(output): 
            print('\ndiff = ' + str(diff))
            print('Current policy')
            print(current_policy)
            print('With state values')
            print(state_values.reshape(5,5))
        
        delta = min([delta, np.sum(np.abs(np.subtract(v, state_values)))])
        v = np.copy(state_values) ## copy of state values to evaluate next iteration
    return current_policy

policy_iteration(policy, state_to_state_probs, rewards, taboos, gamma = 1.0, output = True)

Examine your results. First look at the state values at convergence of the policy iteration algorithm and answer the following questions:
1. Are non-taboo state values closest to the goal the largest? ANS: yes
2. Are the non-taboo state values farthest from the goal the smallest? Keep in mind the robot must travel around the barrier. ANS: yes
3. Are the non-taboo state values symmetric (e.g. same) with respect to distance from the goal? ANS: Yes.
4. Do the taboo states have 0 values? ANS: Yes.

If your answer to any of the above questions is No, there is an error in your code. 

Next, examine the policy you have computed. Do the follow:
1. Follow the optimal paths from the 4 corners of the grid to the goal. How does the symmetry and length of these paths make sense in terms of length and state values? ANS: 
2. Imagine that the door for the warehouse is at position (state) 2. Insert an illustration showing the paths of the optimal plans below. You are welcome to start with the PowerPoint illustration in the course Github repository.  

<img src="OptimalPlansOnGridWorld.JPG" alt="Drawing" style="width:200px; height:200px"/>
<center> **Grid world optimal plans from state 2 to the goal shown goes here** </center>

> **Jagriti: the above is a sample answer.** 

## Value Iteration 

Finally, your will use the value iteration algorithm to compute an optimal policy for the robot reaching the goal. Keep in mind that you will need to maintain a state value of 0 for the taboo states. 

Compare your results from the value iteration algorithm to your results from the policy iteration algorithm and answer the following questions:
1. Are the state values identical between the two methods? ANS: 
2. Ignoring the taboo states, are the plans computed by the two methods identical? ANS: 